# 🚀 **Advanced Data Analysis Assignment**

Welcome to the next-level assignment! We’ll build on the two previous datasets:
1. A **region-based** dataset containing `Region`, `Sales`, and `Transactions`.
2. A **time-series** dataset containing daily `Sales` from 2020-01-01 to 2020-12-31.

In this notebook, you will:
1. Load and explore both datasets.
2. Perform advanced grouping and pivoting on the regional data.
3. Check correlations and detect potential outliers.
4. Conduct advanced time-series analysis (rolling means & seasonal decomposition).
5. Provide concise insights from your findings.

Let's get started! 🎉


## 🧩 **Part A: Advanced Analysis on Regional Sales Data**
We'll begin by re-generating (or reloading) the regional sales data from your previous assignment.

In [2]:
# === Part A: Data Generation (Regional) ===
import pandas as pd
import numpy as np

# Seed for reproducibility
np.random.seed(0)

# Generate random data
data_regional = {
    'Region': np.random.choice(['North', 'South', 'East', 'West'], size=100),
    'Sales': np.random.rand(100) * 1000,  # Sales figures between 0 and 1000
    'Transactions': np.random.randint(1, 100, size=100)  # Transactions between 1 and 100
}

# Create DataFrame
df_regional = pd.DataFrame(data_regional)
df_regional.head()

,Region,Sales,Transactions
0,North,570.196770,29
1,West,438.601513,3
2,South,988.373838,28
3,North,102.044811,84
4,West,208.876756,90


### 🔍 **Task A1: Exploratory Data Analysis**
1. Display basic summary statistics for `Sales` and `Transactions`.
2. Identify the number of unique regions.
3. Check for any missing values.


In [3]:
# === SOLUTION for Task A1 ===

# 1) Basic summary statistics
print(df_regional.describe())
# 2) Number of unique regions
print('\nNumber of unique regions:', len(pd.unique(df_regional['Region'])))
# 3) Check for missing values
missing_count = df_regional.isnull().sum()

print('\nMissing values:')
print(missing_count)

            Sales  Transactions
count  100.000000    100.000000
mean   496.438899     48.440000
std    283.716158     28.051655
min      4.695476      1.000000
25%    262.365019     28.750000
50%    544.924754     44.500000
75%    700.581602     72.500000
max    998.847007     98.000000

Number of unique regions: 4

Missing values:
Region          0
Sales           0
Transactions    0
dtype: int64


### 💹 **Task A2: Pivot Table & Group Analysis**
1. Create a pivot table showing the **average Sales** and **average Transactions** by `Region`.
2. Sort the pivot table by the highest average Sales.


In [4]:
# === SOLUTION for Task A2 ===
# Sort by highest average Sales
regions_avg = df_regional[['Region', 'Sales', 'Transactions']].groupby('Region').mean()
pivot_df = df_regional.pivot_table(
    index='Region',
    values=['Sales', 'Transactions'],
    aggfunc='mean'
)
print(pivot_df)

             Sales  Transactions
Region                          
East    564.093444     43.684211
North   515.117684     47.320000
South   466.730246     44.708333
West    463.957703     54.937500


### ⚗️ **Task A3: Correlation & Outlier Detection** ⚠️ Optional Challenge
1. Calculate the correlation between `Sales` and `Transactions`. Do they appear to be correlated?
2. Detect potential outliers in `Sales` using the **IQR** (Interquartile Range) method.


In [6]:
# === SOLUTION for Task A3 ===
# 1) Correlation
corr_value = df_regional['Sales'].corr(df_regional['Transactions'])
print("Correlation between Sales and Transactions:", corr_value)

# 2) Outlier Detection using IQR
Q1 = df_regional['Sales'].quantile(0.25)
Q3 = df_regional['Sales'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print("Interquartile Range (IQR): ", IQR)
print("Lower Bound: ", lower_bound)
print("Upper Bound: ", upper_bound)

outliers = df_regional[(df_regional['Sales'] < lower_bound) | (df_regional['Sales'] > upper_bound)]
print("\nNumber of outliers in 'Sales': ", len(outliers))
outliers


Correlation between Sales and Transactions: -0.017727723478534588
Interquartile Range (IQR):  438.2165835336902
Lower Bound:  -394.9598564610057
Upper Bound:  1357.906477673755

Number of outliers in 'Sales':  0


,Region,Sales,Transactions


---
## 📈 **Part B: Advanced Time-Series Analysis**
Now let's work with the **time-series** dataset from your second assignment. We'll generate (or reload) the data below.

In [94]:
# === Part B: Data Generation (Time-Series) ===
dates = pd.date_range(start="2020-01-01", end="2020-12-31", freq="D")
data_timeseries = {
    "Date": dates,
    "Sales": (
        np.random.rand(len(dates)) * 200
        + np.sin(np.linspace(-3, 3, len(dates))) * 50
        + 100
    ),
}

df_timeseries = pd.DataFrame(data_timeseries)
# df_timeseries.set_index("Date", inplace=True)

### 🔎 **Task B1: Quick Exploration**
1. Display the first 5 rows.
2. Show a statistical summary of the `Sales` column.

In [11]:
# === SOLUTION for Task B1 ===
# 1) Display first 5 rows of df_timeseries
df_timeseries.head()

,Date,Sales
0,2020-01-01,184.146924
1,2020-01-02,273.414915
2,2020-01-03,118.764805
3,2020-01-04,136.356354
4,2020-01-05,266.023902


In [42]:
# 2) Statistical summary of the 'Sales' column
df_timeseries['Sales'].describe()

count    366.000000
mean     203.849782
std       68.809135
min       50.719030
25%      153.098856
50%      199.144675
75%      251.609671
max      346.521878
Name: Sales, dtype: float64

### 📆 **Task B2: Monthly & Rolling Analysis**
1. Calculate monthly average `Sales`.
2. Compute a 7-day rolling average to smooth out short-term fluctuations.


In [96]:
# === SOLUTION for Task B2 ===
# 1) Monthly average Sales
# df_timeseries['Month'] = df_timeseries.index.to_period('M')
# monthly_avg_sales = df_timeseries.groupby('Month')['Sales'].mean()
df_timeseries["Date"] = pd.to_datetime(df_timeseries["Date"])
df_timeseries.set_index("Date", inplace=True)
monthly_avg_sales = df_timeseries.resample("ME")['Sales'].mean()
df_timeseries.reset_index(inplace=True)
print(monthly_avg_sales)

Date
2020-01-31    176.799319
2020-02-29    162.284615
2020-03-31    133.045222
2020-04-30    154.459183
2020-05-31    189.133565
2020-06-30    197.910887
2020-07-31    216.853578
2020-08-31    233.917748
2020-09-30    241.831597
2020-10-31    256.118151
2020-11-30    224.213604
2020-12-31    242.393320
Freq: ME, Name: Sales, dtype: float64


In [14]:
# 2) 7-day rolling average
df_timeseries['rolling-7-day-avg'] = df_timeseries['Sales'].rolling(window=7).mean()
print(df_timeseries)

                 Sales  rolling-7-day-avg
Date                                     
2020-01-01  184.146924                NaN
2020-01-02  273.414915                NaN
2020-01-03  118.764805                NaN
2020-01-04  136.356354                NaN
2020-01-05  266.023902                NaN
...                ...                ...
2020-12-27  173.590761         160.420850
2020-12-28  158.794469         163.213619
2020-12-29  221.235904         159.045189
2020-12-30  289.139548         175.935165
2020-12-31  223.077383         190.476518

[366 rows x 2 columns]


### 🔬 **Task B3: Day-of-Week Seasonality Analysis (Using Pandas Only)**

1. **Extract the day of the week** from the index and store it in a new column (e.g., `DayOfWeek`).
2. **Group by** this `DayOfWeek` column to get the **average Sales** for each day of the week.
3. **Compare** these daily averages to see if certain days have higher or lower sales.


In [83]:
# === SOLUTION for Task B3 with Pandas Only ===

# 1) Extract day of the week: Monday=0, Sunday=6
df_timeseries['DayOfWeek'] = df_timeseries['Date'].dt.day_name()
# 2) Group by the day of the week to compute average sales
day_of_week_avg = df_timeseries.groupby('DayOfWeek')['Sales'].mean()
print(day_of_week_avg)

DayOfWeek
Friday       199.116197
Monday       204.653854
Saturday     189.369580
Sunday       189.386770
Thursday     210.533559
Tuesday      196.933595
Wednesday    211.916607
Name: Sales, dtype: float64


### 📝 **Observations & Insights**
1. **Regional Data**
   - The correlation between `Sales` and `Transactions` is quite low, suggesting they’re not strongly related in this sample.
   - Pivot tables show which region averages the highest Sales, with minimal outliers in `Sales`.

2. **Time-Series Data**
   - The monthly averages reveal slight fluctuations each month.
   - The 7-day rolling average smooths out daily noise.
   - Seasonal decomposition indicates a clear weekly seasonal pattern (due to the `np.sin()` component) and an overall trend.

---
## 🏁 **Assignment Wrap-Up**

🎉 **Congratulations!** You’ve:
- Built pivot tables and looked for regional trends.
- Analyzed correlation and outliers.
- Explored monthly averages in time-series data.
- Investigated rolling averages and seasonal decomposition.

These techniques will provide a solid foundation for more advanced analytical work, including forecasting, anomaly detection, and deeper business intelligence. Keep exploring!
